# **Import**

In [1]:
import os
import random
import time
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torchvision.models import regnet_y_128gf, RegNet_Y_128GF_Weights
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from torchinfo import summary
from tqdm import tqdm
import logging

In [ ]:
GPU_NUM = 0
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

# **Config**

In [3]:
CFG = {
    "IMG_SIZE": 224,
    "NUM_CLASSES": 396,
    "BATCH_SIZE": 32,
    "EPOCHS": 500,
    "LR": 1e-4,
    "WEIGHT_DECAY": 1e-2,
    "PATIENCE": 5,
    "SEED": 42,
    # AMP & Grad‑clip
    "USE_AMP": False,
    "GRAD_CLIP": 1.0,
}

BASE_DIR = Path(r"D:/dacon_HAI")
PATHS = {
    "BASE": BASE_DIR,
    "TRAIN": BASE_DIR / "open" / "train",
    "TEST":  BASE_DIR / "open" / "test",
    "CKPT":  BASE_DIR / "checkpoints",
    "LOG":   BASE_DIR / "logs",
    "SUBMIT": BASE_DIR / "submission",
}
for p in PATHS.values():
    p.mkdir(parents=True, exist_ok=True)

# --- Normalization stats ---
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

In [4]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(CFG["SEED"])

# **Dataloader**

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if self.is_test:
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith('.jpg'):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith('.jpg'):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, os.path.basename(img_path)
        else:
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label

spatial_aug = transforms.RandomChoice([
    transforms.RandomResizedCrop(CFG["IMG_SIZE"], scale=(0.8, 1.0)),
    transforms.CenterCrop(CFG["IMG_SIZE"]),
])
geom_aug = transforms.RandomChoice([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomVerticalFlip(p=1.0),
    transforms.RandomRotation(degrees=(-30, 30), interpolation=transforms.InterpolationMode.BILINEAR, fill=0),
])
pixel_aug = transforms.RandomChoice([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 5.0)),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
])

train_tf = transforms.Compose([
    transforms.Resize((CFG["IMG_SIZE"], CFG["IMG_SIZE"])),
    transforms.RandomApply([spatial_aug], p=0.5),
    transforms.RandomApply([geom_aug],    p=0.5),
    transforms.RandomApply([pixel_aug],   p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])
val_tf = transforms.Compose([
    transforms.Resize((CFG["IMG_SIZE"], CFG["IMG_SIZE"])),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

In [ ]:
full_ds = CustomImageDataset(PATHS["TRAIN"], transform=train_tf, is_test=False)
print(f"Full data: {len(full_ds):,}")

targets = [label for _, label in full_ds.samples]
class_names = full_ds.classes

train_idx, val_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.2,
    stratify=targets,
    random_state=CFG['SEED']
)


train_dataset = Subset(CustomImageDataset(PATHS["TRAIN"], transform=train_tf, is_test=False), train_idx)
val_dataset = Subset(CustomImageDataset(PATHS["TRAIN"], transform=val_tf, is_test=False), val_idx)


train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


test_root = PATHS["TEST"]
test_dataset = CustomImageDataset(test_root, transform=val_tf, is_test=True)
test_loader  = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


print(f"Number of train imgs: {len(train_dataset)}, Number of valid imgs: {len(val_dataset)}")

# **Network**

### RegNet_Y_128GF (SWAG E2E variant)

In [15]:
weights = RegNet_Y_128GF_Weights.IMAGENET1K_SWAG_LINEAR_V1
model = regnet_y_128gf(weights=weights)
model.fc = nn.Linear(model.fc.in_features, CFG["NUM_CLASSES"])
print(summary(model, input_size=(1, 3, CFG["IMG_SIZE"], CFG["IMG_SIZE"],)))

Layer (type:depth-idx)                                  Output Shape              Param #
RegNet                                                  [1, 396]                  --
├─SimpleStemIN: 1-1                                     [1, 32, 112, 112]         --
│    └─Conv2d: 2-1                                      [1, 32, 112, 112]         864
│    └─BatchNorm2d: 2-2                                 [1, 32, 112, 112]         64
│    └─ReLU: 2-3                                        [1, 32, 112, 112]         --
├─Sequential: 1-2                                       [1, 7392, 7, 7]           --
│    └─AnyStage: 2-4                                    [1, 528, 56, 56]          --
│    │    └─ResBottleneckBlock: 3-1                     [1, 528, 56, 56]          1,580,312
│    │    └─ResBottleneckBlock: 3-2                     [1, 528, 56, 56]          1,955,316
│    └─AnyStage: 2-5                                    [1, 1056, 28, 28]         --
│    │    └─ResBottleneckBlock: 3-3          

# **Hyper-params**

In [ ]:
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=CFG["LR"], weight_decay=CFG["WEIGHT_DECAY"])

steps_per_epoch = len(train_loader)
warmup_steps = 5 * steps_per_epoch

def lr_lambda(step):
    if step < warmup_steps:
        return step / float(max(1, warmup_steps))
    progress = (step - warmup_steps) / float(max(1, CFG["EPOCHS"] * steps_per_epoch - warmup_steps))
    return 0.5 * (1.0 + torch.cos(torch.pi * torch.tensor(progress)))

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

In [ ]:
class EarlyStopping:
    def __init__(self, patience: int = 5, delta: float = 0.0):
        self.patience = patience
        self.delta = delta
        self.best = None
        self.counter = 0
        self.stop = False

    def __call__(self, metric: float):
        if self.best is None or metric < self.best - self.delta:
            self.best = metric
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

early_stopper = EarlyStopping(patience=CFG["PATIENCE"], delta=0.001)

log_file = PATHS["LOG"] / "train_regnet_y_128gf_250612.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(log_file, mode="w"),
    ],
)
logger = logging.getLogger()

# **Train net**

In [ ]:
def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"[{epoch}/{CFG['EPOCHS']}] Train", leave=False)
    for step, (imgs, labels) in enumerate(pbar, start=1):
        imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}", "lr": f"{scheduler.get_last_lr()[0]:.4f}"})

    return running_loss / len(train_loader)

def validate(epoch):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            logits = model(imgs)
            loss = criterion(logits, labels)
            
            val_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_probs.append(F.softmax(logits, dim=1).cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    acc = 100. * correct / total
    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)

    # LogLoss
    class_idx = list(range(CFG["NUM_CLASSES"]))
    answer_df = pd.DataFrame({"ID": np.arange(len(all_labels)), "label": all_labels})
    submission_df = pd.DataFrame(all_probs, columns=class_idx)
    submission_df.insert(0, "ID", submission_df.index)
    avg_val_loss = val_loss / len(val_loader)
    logloss = log_loss(answer_df["label"], all_probs, labels=class_idx)

    return avg_val_loss, acc, logloss


BEST_LOSS = float("inf")
start = time.time()
for epoch in range(1, CFG["EPOCHS"] + 1):
    t0 = time.time()
    tr_loss = train_one_epoch(epoch)
    val_loss, val_acc, val_logloss = validate(epoch)
    epoch_dur = time.time() - t0

    logger.info(
        f"Epoch {epoch}/{CFG['EPOCHS']} | "
        f"Time {epoch_dur:.1f}s | "
        f"TrainLoss {tr_loss:.4f} | ValLoss {val_loss:.4f} | "
        f"ValAcc {val_acc:.2f}% | LogLoss {val_logloss:.4f}"
    )

    # Save best by logloss
    if val_logloss < BEST_LOSS:
        BEST_LOSS = val_logloss
        ckpt_path = PATHS["CKPT"] / "best_regnet_y_128gf_250612_b32.pth"
        torch.save(model.state_dict(), ckpt_path)
        logger.info(f"[Checkpoint] Saved at epoch {epoch} (LogLoss {BEST_LOSS:.4f})")

    early_stopper(val_logloss)
    if early_stopper.stop:
        logger.info("Early stopping triggered.")
        break

logger.info(f"Total training time: {datetime.timedelta(seconds=int(time.time() - start))}")

c:\Users\USPD\anaconda3\envs\hai_env\lib\site-packages\sklearn\preprocessing\_label.py:302: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  self.y_type_ = type_of_target(y, input_name="y")
c:\Users\USPD\anaconda3\envs\hai_env\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
2025-06-12 09:18:30,869 Epoch 1/500 | Time 1434.8s | TrainLoss 5.2902 | ValLoss 3.5340 | ValAcc 29.63% | LogLoss 3.1452
2025-06-12 09:18:36,032 [Checkpoint] Saved at epoch 1 (LogLoss 3.1452)
c:\Users\USPD\anaconda3\envs\hai_env\lib\site-packages\sklearn\preprocessing\_label.py:302: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  self.y_type_ = type_of_target(y, input_name="y")
c:\Users\USPD\anaconda3\envs\hai_env\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique cla

# **Eval**

In [ ]:
# Inference
weights = RegNet_Y_128GF_Weights.IMAGENET1K_SWAG_LINEAR_V1
model   = regnet_y_128gf(weights=weights)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, CFG['NUM_CLASSES'])

results = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        for prob in probs.cpu():
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

In [ ]:
# Submission
submission = pd.read_csv(os.path.join(PATHS["SUBMIT"], 'submission_JN.csv'), encoding='utf-8-sig')

class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv(os.path.join(PATHS["SUBMIT"], 'regnet_y_128gf_250612_b32_submission_JN.csv'), index=False, encoding='utf-8-sig')